cargo los datos del drive

In [1]:
from google.colab import drive
drive.mount('/content/drive');

Mounted at /content/drive


In [2]:
# importamos librerias para el manejo de datos pandas para manejo de datos y numpy operaciones con numeros
import numpy as np
import pandas as pd

#import tools

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
#guardo el train_1 como csv, el delimitado por defecto es coma y le quito la columna index
#train_1.to_csv('/content/drive/MyDrive/bigdata3/proyecto homicidios/columnas_necesarias.csv',index=False)

In [12]:
train=pd.read_csv('/content/drive/MyDrive/bigdata3/proyecto homicidios/columnas_necesarias.csv')

In [13]:
train.head()

,Fecha,Departamento,Dia,HoraEstandarizada,Zona,Modalidad,Movil_Agresor,Movil_Victima,Edad,Sexo
0,08/12/2010 12:00:00 AM,VALLE,Miercoles,1,URBANA,ARMA DE FUEGO,A PIE,A PIE,18,MASCULINO
1,01/01/2010 12:00:00 AM,ANTIOQUIA,Viernes,8,URBANA,ARMA DE FUEGO,A PIE,A PIE,32,MASCULINO
2,08/11/2010 12:00:00 AM,NORTE DE SANTANDER,Lunes,7,URBANA,ARMA DE FUEGO,CONDUCTOR MOTOCICLETA,A PIE,48,MASCULINO
3,2010-01-01 00:00:00,CAUCA,Lunes,23,URBANA,ARMA DE FUEGO,A PIE,A PIE,26,MASCULINO
4,2010-01-01 00:00:00,CUNDINAMARCA,Domingo,0,URBANA,ARMA BLANCA,A PIE,A PIE,43,MASCULINO


In [14]:
train_1 = train[['Fecha', 'Departamento', 'Dia', 'HoraEstandarizada', 'Modalidad', 'Movil_Agresor', 'Movil_Victima', 'Zona', 'Edad', 'Sexo']]
train_1.head()

,Fecha,Departamento,Dia,HoraEstandarizada,Modalidad,Movil_Agresor,Movil_Victima,Zona,Edad,Sexo
0,08/12/2010 12:00:00 AM,VALLE,Miercoles,1,ARMA DE FUEGO,A PIE,A PIE,URBANA,18,MASCULINO
1,01/01/2010 12:00:00 AM,ANTIOQUIA,Viernes,8,ARMA DE FUEGO,A PIE,A PIE,URBANA,32,MASCULINO
2,08/11/2010 12:00:00 AM,NORTE DE SANTANDER,Lunes,7,ARMA DE FUEGO,CONDUCTOR MOTOCICLETA,A PIE,URBANA,48,MASCULINO
3,2010-01-01 00:00:00,CAUCA,Lunes,23,ARMA DE FUEGO,A PIE,A PIE,URBANA,26,MASCULINO
4,2010-01-01 00:00:00,CUNDINAMARCA,Domingo,0,ARMA BLANCA,A PIE,A PIE,URBANA,43,MASCULINO


In [15]:
#validamos de nuevo si quedaron valores nulos en los datos
print('train')
print(pd.isnull(train_1).sum())

train
Fecha                0
Departamento         0
Dia                  0
HoraEstandarizada    0
Modalidad            0
Movil_Agresor        0
Movil_Victima        0
Zona                 0
Edad                 0
Sexo                 0
dtype: int64


In [16]:
#para la columna fecha
#convierto la columna Fecha a tipo fecha con to_datetime()
df = pd.to_datetime(train_1['Fecha'])
#cargo el train_1 en un dataframe para agregar la columna mes
train_1 = pd.DataFrame(train_1)
#agrego la columna mes al train_1
train_1['Mes'] = df.dt.month

#ya se obtubo el mes de la fecha y se agregó esa nueva columna al train_1
#creamos categorias a paetir de los valores existentes para el mes
train_1=pd.concat([train_1,pd.get_dummies(train_1['Mes'],prefix='Mes_')],axis=1)
#miro como van quedando los datos

#hacemos lo mismo para la columna Departamento
train_1=pd.concat([train_1,pd.get_dummies(train_1['Departamento'],prefix='Departamento_')],axis=1)

#hacemos lo mismo para la columna Dia
train_1=pd.concat([train_1,pd.get_dummies(train_1['Dia'],prefix='Dia_')],axis=1)

#para la columna HoraEstandarizada que se sacó solo la hora mas rápido con excel
train_1=pd.concat([train_1,pd.get_dummies(train_1['HoraEstandarizada'],prefix='Hora_')],axis=1)

#dummies crea categorias a paetir de los valores existentes para Zona
train_1=pd.concat([train_1,pd.get_dummies(train_1['Zona'],prefix='Zona_')],axis=1)

#hacemos lo mismo para la columna Modalidad
train_1=pd.concat([train_1,pd.get_dummies(train_1['Modalidad'],prefix='Modalidad_')],axis=1)

#hacemos lo mismo para la columna Movil_Agresor
train_1=pd.concat([train_1,pd.get_dummies(train_1['Movil_Agresor'],prefix='Movil_Agresor_')],axis=1)

#hacemos lo mismo para la columna Movil_Victima
train_1=pd.concat([train_1,pd.get_dummies(train_1['Movil_Victima'],prefix='Movil_Victima_')],axis=1)

#normalizo la columna edad (valor - min) / (max - min)
df_norm = (train_1['Edad'] - train_1['Edad'].min()) / (train_1['Edad'].max()-train_1['Edad'].min())
#ya habia agregado el train_1 a un data frame en fecha ahora agrego la columna edad normalizada al train_1
train_1['edad_normalizada'] = df_norm

#hacemos lo mismo para la columna sexo
#cambio el texto por 0 mujer y 1 hombre
train_1['Sexo'].replace(
   to_replace= ['FEMENINO','MASCULINO'],
    value= [0,1],
    inplace=True
)


train_1.head()

,Fecha,Departamento,Dia,HoraEstandarizada,Modalidad,Movil_Agresor,Movil_Victima,Zona,Edad,Sexo,Mes,Mes__1,Mes__2,Mes__3,Mes__4,Mes__5,Mes__6,Mes__7,Mes__8,Mes__9,Mes__10,Mes__11,Mes__12,Departamento__AMAZONAS,Departamento__ANTIOQUIA,Departamento__ARAUCA,Departamento__ATLANTICO,Departamento__BOLIVAR,Departamento__BOYACA,Departamento__CALDAS,Departamento__CAQUETA,Departamento__CASANARE,Departamento__CAUCA,Departamento__CESAR,Departamento__CHOCO,Departamento__CORDOBA,Departamento__CUNDINAMARCA,Departamento__GUAINIA,Departamento__GUAJIRA,Departamento__GUAVIARE,...,Modalidad__LIQUIDOS,Modalidad__MINA ANTIPERSONA,Modalidad__MOTO BOMBA,Modalidad__OLLA BOMBA,Modalidad__PAQUETE BOMBA,Modalidad__PERSONA BOMBA,Modalidad__POLVORA(FUEGOS PIROTECNICOS),Modalidad__PRENDAS DE VESTIR,Modalidad__PUNZANTES,Modalidad__QUIMICOS,Modalidad__ROCKET,Modalidad__SIN EMPLEO DE ARMAS,Modalidad__SUSTANCIAS TOXICAS,Modalidad__VENENO,Movil_Agresor__A PIE,Movil_Agresor__BICICLETA,Movil_Agresor__CONDUCTOR BUS,Movil_Agresor__CONDUCTOR MOTOCICLETA,Movil_Agresor__CONDUCTOR TAXI,Movil_Agresor__CONDUCTOR VEHICULO,Movil_Agresor__PASAJERO AERONAVE,Movil_Agresor__PASAJERO BARCO,Movil_Agresor__PASAJERO BUS,Movil_Agresor__PASAJERO MOTOCICLETA,Movil_Agresor__PASAJERO TAXI,Movil_Agresor__PASAJERO VEHICULO,Movil_Agresor__VEHICULO,Movil_Victima__A PIE,Movil_Victima__BICICLETA,Movil_Victima__CONDUCTOR BUS,Movil_Victima__CONDUCTOR MOTOCICLETA,Movil_Victima__CONDUCTOR TAXI,Movil_Victima__CONDUCTOR VEHICULO,Movil_Victima__PASAJERO BARCO,Movil_Victima__PASAJERO BUS,Movil_Victima__PASAJERO MOTOCICLETA,Movil_Victima__PASAJERO TAXI,Movil_Victima__PASAJERO VEHICULO,Movil_Victima__VEHICULO,edad_normalizada
0,08/12/2010 12:00:00 AM,VALLE,Miercoles,1,ARMA DE FUEGO,A PIE,A PIE,URBANA,18,1,8,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.18
1,01/01/2010 12:00:00 AM,ANTIOQUIA,Viernes,8,ARMA DE FUEGO,A PIE,A PIE,URBANA,32,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.32
2,08/11/2010 12:00:00 AM,NORTE DE SANTANDER,Lunes,7,ARMA DE FUEGO,CONDUCTOR MOTOCICLETA,A PIE,URBANA,48,1,8,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.48
3,2010-01-01 00:00:00,CAUCA,Lunes,23,ARMA DE FUEGO,A PIE,A PIE,URBANA,26,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.26
4,2010-01-01 00:00:00,CUNDINAMARCA,Domingo,0,ARMA BLANCA,A PIE,A PIE,URBANA,43,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.43


In [17]:
#borro las columnas que categorice con drop, el implace me dice que ejecuta todo sobre la misma 
# variable, es equivalente a train_1=train_1
train_1.drop(labels=['Fecha'],axis=1,inplace=True)
train_1.drop(labels=['Departamento'],axis=1,inplace=True)
train_1.drop(labels=['Dia'],axis=1,inplace=True)
train_1.drop(labels=['HoraEstandarizada'],axis=1,inplace=True)
train_1.drop(labels=['Zona'],axis=1,inplace=True)
train_1.drop(labels=['Modalidad'],axis=1,inplace=True)
train_1.drop(labels=['Movil_Agresor'],axis=1,inplace=True)
train_1.drop(labels=['Movil_Victima'],axis=1,inplace=True)
train_1.drop(labels=['Edad'],axis=1,inplace=True)
#train_1.drop(labels=['Sexo'],axis=1,inplace=True)
train_1.drop(labels=['Mes'],axis=1,inplace=True)
train_1.head()


,Sexo,Mes__1,Mes__2,Mes__3,Mes__4,Mes__5,Mes__6,Mes__7,Mes__8,Mes__9,Mes__10,Mes__11,Mes__12,Departamento__AMAZONAS,Departamento__ANTIOQUIA,Departamento__ARAUCA,Departamento__ATLANTICO,Departamento__BOLIVAR,Departamento__BOYACA,Departamento__CALDAS,Departamento__CAQUETA,Departamento__CASANARE,Departamento__CAUCA,Departamento__CESAR,Departamento__CHOCO,Departamento__CORDOBA,Departamento__CUNDINAMARCA,Departamento__GUAINIA,Departamento__GUAJIRA,Departamento__GUAVIARE,Departamento__HUILA,Departamento__MAGDALENA,Departamento__META,Departamento__NARINO,Departamento__NORTE DE SANTANDER,Departamento__PUTUMAYO,Departamento__QUINDIO,Departamento__RISARALDA,Departamento__SAN ANDRES,Departamento__SANTANDER,...,Modalidad__LIQUIDOS,Modalidad__MINA ANTIPERSONA,Modalidad__MOTO BOMBA,Modalidad__OLLA BOMBA,Modalidad__PAQUETE BOMBA,Modalidad__PERSONA BOMBA,Modalidad__POLVORA(FUEGOS PIROTECNICOS),Modalidad__PRENDAS DE VESTIR,Modalidad__PUNZANTES,Modalidad__QUIMICOS,Modalidad__ROCKET,Modalidad__SIN EMPLEO DE ARMAS,Modalidad__SUSTANCIAS TOXICAS,Modalidad__VENENO,Movil_Agresor__A PIE,Movil_Agresor__BICICLETA,Movil_Agresor__CONDUCTOR BUS,Movil_Agresor__CONDUCTOR MOTOCICLETA,Movil_Agresor__CONDUCTOR TAXI,Movil_Agresor__CONDUCTOR VEHICULO,Movil_Agresor__PASAJERO AERONAVE,Movil_Agresor__PASAJERO BARCO,Movil_Agresor__PASAJERO BUS,Movil_Agresor__PASAJERO MOTOCICLETA,Movil_Agresor__PASAJERO TAXI,Movil_Agresor__PASAJERO VEHICULO,Movil_Agresor__VEHICULO,Movil_Victima__A PIE,Movil_Victima__BICICLETA,Movil_Victima__CONDUCTOR BUS,Movil_Victima__CONDUCTOR MOTOCICLETA,Movil_Victima__CONDUCTOR TAXI,Movil_Victima__CONDUCTOR VEHICULO,Movil_Victima__PASAJERO BARCO,Movil_Victima__PASAJERO BUS,Movil_Victima__PASAJERO MOTOCICLETA,Movil_Victima__PASAJERO TAXI,Movil_Victima__PASAJERO VEHICULO,Movil_Victima__VEHICULO,edad_normalizada
0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.18
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.32
2,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.48
3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.26
4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.43


In [18]:
# Y = mX + b  :: Y = wX + b
y = train_1['Sexo']
print(y)

0         1
1         1
2         1
3         1
4         1
         ..
102300    1
102301    1
102302    0
102303    1
102304    1
Name: Sexo, Length: 102305, dtype: int64


In [19]:
x = train_1
x.drop('Sexo', axis=1, inplace=True)
print(x)

        Mes__1  Mes__2  ...  Movil_Victima__VEHICULO  edad_normalizada
0            0       0  ...                        0              0.18
1            1       0  ...                        0              0.32
2            0       0  ...                        0              0.48
3            1       0  ...                        0              0.26
4            1       0  ...                        0              0.43
...        ...     ...  ...                      ...               ...
102300       0       0  ...                        0              0.13
102301       0       0  ...                        0              0.19
102302       0       0  ...                        0              0.28
102303       0       0  ...                        0              0.34
102304       0       0  ...                        0              0.19

[102305 rows x 140 columns]


In [20]:
# Separar los datos
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.176)

In [21]:
print('== Algunos clasificadores ==')
#print('Regresión logística')
logReg = LogisticRegression()
logReg.fit(x_train, y_train)
y_predict = logReg.predict(x_val)

print('Exactitud LogisticRegression (train)', logReg.score(x_train, y_train))
print('Exactitud LogisticRegression (test)', logReg.score(x_test, y_test))

print('####prediccion########')
#cambio el  o y 1 por texto
y_predict=pd.DataFrame(y_predict)
y_predict.rename(columns={0:'prediccion'},inplace=True)
y_predict.replace(
   to_replace= [0,1],
    value= ['FEMENINO','MASCULINO'],
    inplace=True
)
print(y_predict.head(10))
print('####validacion########')
#cambio el  o y 1 por texto
y_val=pd.DataFrame(y_val)
y_val.rename(columns={'Sexo':'validacion'},inplace=True)
y_val.replace(
   to_replace= [0,1],
    value= ['FEMENINO','MASCULINO'],
    inplace=True
)
print(y_val.head(10))

== Algunos clasificadores ==
Exactitud LogisticRegression (train) 0.9165294331091076
Exactitud LogisticRegression (test) 0.9152873713019679
####prediccion########
  prediccion
0  MASCULINO
1  MASCULINO
2  MASCULINO
3  MASCULINO
4  MASCULINO
5  MASCULINO
6  MASCULINO
7  MASCULINO
8  MASCULINO
9  MASCULINO
####validacion########
      validacion
88649  MASCULINO
68806  MASCULINO
65567  MASCULINO
56678   FEMENINO
5776   MASCULINO
34896  MASCULINO
73138  MASCULINO
21615  MASCULINO
97936  MASCULINO
62442  MASCULINO


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
#guardo el modelo
import pickle
rlfile = open("/content/drive/MyDrive/bigdata3/modelos/modelo_generoLoR.pickle", 'wb')
pickle.dump(logReg,rlfile)
rlfile.close()

In [ ]:
y_predictloR = logReg.predict(x_val)
#grafica para el modelo logReg
import matplotlib.pyplot as plt
plt.plot(y_predictloR, label='Predicted')
plt.plot(y_test.values, label='Expected')
plt.title("Género LogisticRegression")
plt.legend()
plt.show()

In [ ]:
#ahora con ('KNN')
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
y_predict = knn.predict(x_val)
print('Precisión KNN (train)', knn.score(x_train, y_train))
print('Precisión KNN (test)', knn.score(x_test, y_test))

In [ ]:
y_predicknc = knn.predict(x_val)
#grafica para el modelo logReg
import matplotlib.pyplot as plt
plt.plot(y_predicknc, label='Predicted')
plt.plot(y_test, label='Expected')
plt.title("Género KNeighborsClassifier")
plt.legend()
plt.show()

In [ ]:
#guardo el modelo
import pickle
rlfile = open("/content/drive/MyDrive/bigdata3/modelos/modelo_generoKnn.pickle", 'wb')
pickle.dump(knn,rlfile)
rlfile.close()